1. IMPORT LIBRARIES

In [1]:
!pip install -q datasets==2.16.1
!pip install -q evaluate==0.4.1
!pip install -q transformers[sentencepiece]==4.40.0
!pip install -q accelerate==0.26.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.4.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf

In [2]:
import pandas as pd
import os
import math
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments, get_linear_schedule_with_warmup,EarlyStoppingCallback, pipeline
from huggingface_hub import notebook_login
from datasets import Dataset, load_metric
from sklearn import preprocessing
import torch
import string 
from torch.nn import Dropout
from torch.optim import AdamW
from sklearn import preprocessing

notebook_login()

2024-05-04 07:52:41.145390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 07:52:41.145505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 07:52:41.308310: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2. EDA

In [27]:
dataset_path = "/kaggle/input/viet-poem-dataset/vietnamese_poem_dataset.csv"
dataset = pd.read_csv(dataset_path)

dataset.head()

,Title,Content,Poetry Collection,URL,Writer
0,Ở lại đi!,Nơi này còn một câu chuyện vui chưa hết\ntiếng...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt
1,Không phải lỗi của hoa hồng vàng,Một ngày tình yêu gõ cửa\ncủa ngôi nhà có hoa ...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt
2,Chỉ những chiếc lá mới biết,Có những năm tháng của chúng ta\nrơi theo mùa ...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt
3,Chưa bao giờ và không bao giờ,Chưa bao giờ và không bao giờ ta biết.\nTrí nh...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt
4,Nếu không muốn đi hết con đường...,Nếu không muốn đi hết con đường...\nThì nên dừ...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt


In [28]:
dataset.tail()

,Title,Content,Poetry Collection,URL,Writer
1256,Những giọt nước mắt mẹ tôi,mẹ tôi vấp phải cái múi nối của con đường bê t...,Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ
1257,Theo anh ra ngoài thành Huế,Chiều theo anh lên chùa Thiên Mụ\nNgắm trời xa...,Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ
1258,Vầng trăng viễn phố,Kể từ tóc gội bờ môi\nVầng trăng viễn phố the...,Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ
1259,Về Huyền Không Sơn Thượng,Đây núi đồi bình yên\nCó tiếng chim dịu hiền\n...,Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ
1260,Vọng phương Nam hành,Gửi Đoàn Trung Hữu\n\nNgười đi hun hút phương ...,Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ


In [29]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261 entries, 0 to 1260
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Title              1261 non-null   object
 1   Content            1261 non-null   object
 2   Poetry Collection  1261 non-null   object
 3   URL                1261 non-null   object
 4   Writer             1261 non-null   object
dtypes: object(5)
memory usage: 49.4+ KB


In [30]:
dataset.describe()

,Title,Content,Poetry Collection,URL,Writer
count,1261,1261,1261,1261,1261
unique,1048,1057,44,1059,8
top,Đời phiêu lãng,"Từ khi đôi má đỏ hây hây,\nEm tập thêu thùa, t...",Lệ Thanh thi tập,https://www.thivien.net/H%C3%A0n-M%E1%BA%B7c-T...,Nguyễn Phong Việt
freq,4,4,122,4,339


In [31]:
dataset['Poetry Collection'].unique()

array(['Đi qua thương nhớ (2012)', 'Từ yêu đến thương (2013)',
       'Sinh ra để cô đơn (2014)', 'Sống một cuộc đời bình thường (2015)',
       'Về đâu những vết thương (2016)', ' ',
       'Sao phải đau đến như vậy (2017)',
       'Chỉ cần tin mình là duy nhất (2018)',
       'Mình sẽ đi cuối đất cùng trời (2019)',
       'Bao nhiêu thương nhớ cho vừa (2020)',
       'Đã đi qua thương nhớ (2021)', 'Lạ lùng sao, đớn đau này (2013)',
       'Hương cây (1968)', 'Mây trắng của đời tôi (1989)',
       'Bầy ong trong đêm sâu (1993)', 'Gửi tới các anh (1998)',
       'Di cảo (2008)', 'Những bông hoa không chết (2008)',
       'Tuyển tập chung', 'Chồi biếc (1963)', 'Hoa dọc chiến hào (1968)',
       'Lời ru trên mặt đất (1978)', 'Chờ trăng (1981)', 'Tự hát (1984)',
       'Không bao giờ là cuối (2011)', 'Thơ thơ (1938)',
       'Gửi hương cho gió (1945)', 'Ngọn quốc kỳ (1945)',
       'Dưới sao vàng (1949)', 'Mẹ con (1954)', 'Sáng (1954)',
       'Ngôi sao (1955)', 'Cầm tay (1962)',
       '

In [32]:
dataset['Writer'].unique()

array(['Nguyễn Phong Việt', 'Nguyễn Thiên Ngân', 'Lưu Quang Vũ',
       'Xuân Quỳnh Nguyễn Thị Xuân Quỳnh',
       'Xuân Diệu Ngô Xuân Diệu, Trảo Nha', 'Nguyễn Nhật Ánh',
       'Hàn Mặc Tử Nguyễn Trọng Trí, Hàn Mạc Tử',
       'Nguyễn Lãm Thắng Lam Thuỵ'], dtype=object)

3.PREPROCESS DATASET

In [33]:
def remove_punctuation(text):
    punctuation = string.punctuation + '...' + '…'  # Thêm dấu ba chấm và dấu chấm lửng vào danh sách dấu câu
    translator = str.maketrans('', '', punctuation)  # Tạo bảng chuyển đổi
    text_without_punctuation = text.translate(translator)  # Sử dụng bảng chuyển đổi để loại bỏ dấu câu
    return text_without_punctuation

dataset['Content'] = dataset['Content'].apply(lambda x: remove_punctuation(x))
print(dataset['Content'][200])

Đâu phải lỗi của người ta
khi trả lại chiếc chìa khoá cho ngôi nhà

Những bước chân từ nay không còn đến lượt mình hỏi sẽ còn đi bao xa
chiếc ba lô cũng chẳng cần ai sắp xếp
mặc áo lệch vai hay móng tay dài bụi bám
chắc người ta tự biết
phải nên làm thế nào

Rời khỏi ngôi nhà mà người từng quyết định bước vào
lúc bỏ đôi giày vẫn mang bên ngoài ngạch cửa
cuộc đời thênh thang nên không ai là không đi tìm một điểm tựa
nhưng cuộc đời đâu chỉ là thương nhớ
nuôi ấm áp từng ngày

Người trả lại chiếc chìa khoá bằng cả hai bàn tay
song trên môi là lặng im khép cửa
không cần biết trái tim người kia ngân lên bao nhịp đổ vỡ
ngưng đọng bao nhiêu giây trong một hơi thở
rồi cứ thế quay lưng

Rồi cứ thế để lại ngôi nhà phía sau cả một bóng tối chập chùng
những đêm trắng triền miên với vạn ngàn câu hỏi
cứ ước gì bình minh bắt đầu vào lúc hoàng hôn về tới
mình chỉ cần thay đồ và mặc vào áo mới
mặc kệ lòng đau nhói
lúc bước ra với đời kia

Còn người ta không ai biết đến bao giờ trở về
hay sẽ là bỏ lại đô

In [34]:
def split_content(content):
    samples = []  # Initialize an empty list to store the samples

    # Split the content into different parts of the poem based on empty lines
    poem_parts = content.split('\n\n')

    # Iterate over each part of the poem
    for poem_part in poem_parts:
        # Split each part into lines
        poem_in_lines = poem_part.split('\n')

        # Check if the number of lines in the part is less than 6
        if len(poem_in_lines) < 10:
            # If it is, append the part (represented as a list of lines) to the samples list
            samples.append(poem_in_lines)

    return samples  # Return the list of samples

# Apply the split_content function to the 'Content' column of the dataset
dataset['Content'] = dataset['Content'].apply(lambda x: split_content(x))
dataset['Content'][0]

[['Nơi này còn một câu chuyện vui chưa hết',
  'tiếng cười chưa dứt',
  'đêm chưa về sáng'],
 ['Nơi này còn cần một người để lau nước mắt',
  'cần một tiếng thở dài đồng cảm',
  'cần một cái lay vai thức tỉnh'],
 ['Nơi này còn những ngày thật ngắn',
  'còn những đêm thật dài',
  'còn những bình minh một người chưa thức giấc',
  'những hoàng hôn một người đã chợp mắt'],
 ['Nơi này còn cần một giọng nói tha thiết',
  'còn cần một tấm lòng thành thật',
  'còn cần những ước mơ viển vông'],
 ['Nơi này còn một giấc ngủ bình yên',
  'còn một cơn gió về thật nhẹ',
  'còn một ánh trăng qua cửa'],
 ['Ở lại đi', 'nơi này còn cần tất cả']]

In [35]:
dataset_exploded = dataset.explode('Content')
dataset_exploded.reset_index(drop=True, inplace=True)
dataset_exploded = dataset_exploded.dropna(subset=['Content'])
dataset_exploded

,Title,Content,Poetry Collection,URL,Writer
0,Ở lại đi!,"[Nơi này còn một câu chuyện vui chưa hết, tiến...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt
1,Ở lại đi!,"[Nơi này còn cần một người để lau nước mắt, cầ...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt
2,Ở lại đi!,"[Nơi này còn những ngày thật ngắn, còn những đ...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt
3,Ở lại đi!,"[Nơi này còn cần một giọng nói tha thiết, còn ...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt
4,Ở lại đi!,"[Nơi này còn một giấc ngủ bình yên, còn một cơ...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt
...,...,...,...,...,...
5973,Vọng phương Nam hành,"[Người đi hun hút phương Nam hành, Chỉ một mìn...",Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ
5974,Vọng phương Nam hành,"[Thì thôi mây trắng bay về núi, Một chén hồ tr...",Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ
5975,Vọng phương Nam hành,"[Phương Nam hề mênh mông, Xa lắc những ngày ta...",Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ
5976,Vọng phương Nam hành,"[Người ạ chiều nay buồn ngợp phố, Ta ngó mưa b...",Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ


In [36]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
dataset_exploded['Label Encoding']= label_encoder.fit_transform(dataset_exploded['Title'])

dataset_exploded

,Title,Content,Poetry Collection,URL,Writer,Label Encoding
0,Ở lại đi!,"[Nơi này còn một câu chuyện vui chưa hết, tiến...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
1,Ở lại đi!,"[Nơi này còn cần một người để lau nước mắt, cầ...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
2,Ở lại đi!,"[Nơi này còn những ngày thật ngắn, còn những đ...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
3,Ở lại đi!,"[Nơi này còn cần một giọng nói tha thiết, còn ...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
4,Ở lại đi!,"[Nơi này còn một giấc ngủ bình yên, còn một cơ...",Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
...,...,...,...,...,...,...
5973,Vọng phương Nam hành,"[Người đi hun hút phương Nam hành, Chỉ một mìn...",Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ,814
5974,Vọng phương Nam hành,"[Thì thôi mây trắng bay về núi, Một chén hồ tr...",Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ,814
5975,Vọng phương Nam hành,"[Phương Nam hề mênh mông, Xa lắc những ngày ta...",Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ,814
5976,Vọng phương Nam hành,"[Người ạ chiều nay buồn ngợp phố, Ta ngó mưa b...",Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ,814


In [37]:
print(dataset_exploded['Content'][1])

['Nơi này còn cần một người để lau nước mắt', 'cần một tiếng thở dài đồng cảm', 'cần một cái lay vai thức tỉnh']


In [38]:
dataset_exploded['Content'] = dataset_exploded['Content'].apply(lambda x: '\n'.join(x))
dataset_exploded

,Title,Content,Poetry Collection,URL,Writer,Label Encoding
0,Ở lại đi!,Nơi này còn một câu chuyện vui chưa hết\ntiếng...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
1,Ở lại đi!,Nơi này còn cần một người để lau nước mắt\ncần...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
2,Ở lại đi!,Nơi này còn những ngày thật ngắn\ncòn những đê...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
3,Ở lại đi!,Nơi này còn cần một giọng nói tha thiết\ncòn c...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
4,Ở lại đi!,Nơi này còn một giấc ngủ bình yên\ncòn một cơn...,Đi qua thương nhớ (2012),https://www.thivien.net/Nguy%E1%BB%85n-Phong-V...,Nguyễn Phong Việt,925
...,...,...,...,...,...,...
5973,Vọng phương Nam hành,Người đi hun hút phương Nam hành\nChỉ một mình...,Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ,814
5974,Vọng phương Nam hành,Thì thôi mây trắng bay về núi\nMột chén hồ trư...,Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ,814
5975,Vọng phương Nam hành,Phương Nam hề mênh mông\nXa lắc những ngày ta ...,Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ,814
5976,Vọng phương Nam hành,Người ạ chiều nay buồn ngợp phố\nTa ngó mưa ba...,Đường xưa,https://www.thivien.net/Nguy%E1%BB%85n-L%C3%A3...,Nguyễn Lãm Thắng Lam Thuỵ,814


In [39]:
npv_poem_dataset = Dataset.from_pandas(dataset_exploded)
npv_poem_dataset

Dataset({
    features: ['Title', 'Content', 'Poetry Collection', 'URL', 'Writer', 'Label Encoding', '__index_level_0__'],
    num_rows: 5843
})

In [40]:
#create train_test_split
TEST_SIZE = 0.1
npv_poem_dataset = npv_poem_dataset.train_test_split(test_size=TEST_SIZE)
npv_poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Title', 'Content', 'Poetry Collection', 'URL', 'Writer', 'Label Encoding', '__index_level_0__'],
        num_rows: 5258
    })
    test: Dataset({
        features: ['Title', 'Content', 'Poetry Collection', 'URL', 'Writer', 'Label Encoding', '__index_level_0__'],
        num_rows: 585
    })
})

4. DOWNLOADING PRETRAINED MODEL


In [20]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/gpt-neo-vi-small")
model     = AutoModelForCausalLM.from_pretrained("NlpHUST/gpt-neo-vi-small",device_map="auto")

tokenizer_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/544k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/546M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [21]:
# Thêm dropout vào mô hình
model.config.attention_dropout = 0.2
model.config.hidden_dropout = 0.2
for layer in model.transformer.h:
    layer.attn.dropout = Dropout(0.2)

In [41]:
#Tokenizing
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 200

# Tokenization function
def tokenize_function(row):
    # Tokenize the 'Content' column
    return tokenizer(
        row['Content'],
        max_length=MAX_SEQ_LEN,
        padding='max_length',
        truncation=True
    )

# Tokenize the dataset
tokenized_poem_dataset = npv_poem_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=npv_poem_dataset['train'].column_names)


Map (num_proc=4):   0%|          | 0/5258 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/585 [00:00<?, ? examples/s]

In [23]:
tokenizer('Tuyển tập thơ Việt', max_length=10, padding='max_length', truncation=True)

{'input_ids': [6502, 144, 121, 516, 775, 2312, 1426, 144, 121, 2422], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [42]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/gpt-neo-vi-small")
model     = AutoModelForCausalLM.from_pretrained("NlpHUST/gpt-neo-vi-small",device_map="auto")

In [43]:
from torch.nn import Dropout

# Thêm dropout vào mô hình
model.config.attention_dropout = 0.15
model.config.hidden_dropout = 0.15
for layer in model.transformer.h:
    layer.attn.dropout = Dropout(0.15)

In [47]:
from transformers import TrainingArguments
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup,EarlyStoppingCallback


# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.005)

# Total training steps
total_steps = len(tokenized_poem_dataset['train']) * 250

# Warmup steps (usually 10% of total training steps)
num_warmup_steps = int(0.1 * total_steps)

# Initialize scheduler with warmup
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=total_steps
)

training_args = TrainingArguments(
    output_dir='vi_gpt_poem',
    save_strategy='steps',
    learning_rate=1e-5,
    num_train_epochs=250,
    weight_decay=0.005,
    logging_dir='vi_gpt_poem/logs',
    logging_steps=500,
    evaluation_strategy='steps',
    eval_steps=500,
    per_device_train_batch_size=42,
    per_device_eval_batch_size=42,
    fp16=True,
    push_to_hub=True,
    load_best_model_at_end=True,
    save_total_limit=2,
    seed=42,

)


In [48]:
trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_poem_dataset['train'],
            eval_dataset=tokenized_poem_dataset['test'],
            data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
            tokenizer=tokenizer,
            optimizers=(optimizer, scheduler), 
)

trainer.train()

Step,Training Loss,Validation Loss
500,6.863400,6.190017
1000,4.799900,3.403917
1500,2.747300,2.576648
2000,2.251300,2.205097
2500,1.942600,1.911302
3000,1.705900,1.672312
3500,1.533300,1.519554
4000,1.399600,1.405973
4500,1.306600,1.319296
5000,1.228000,1.251292


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=31500, training_loss=0.8529038647848463, metrics={'train_runtime': 28840.6921, 'train_samples_per_second': 45.578, 'train_steps_per_second': 1.092, 'total_flos': 1.341237224448e+17, 'train_loss': 0.8529038647848463, 'epoch': 250.0})

In [49]:
trainer.push_to_hub(commit_message="Completed Process")

CommitInfo(commit_url='https://huggingface.co/duydatnguyen/vi_gpt_poem_/commit/60bae808c0dbc6530735e3f1b156692024360f6f', commit_message='Completed Process', commit_description='', oid='60bae808c0dbc6530735e3f1b156692024360f6f', pr_url=None, pr_revision=None, pr_num=None)

6. Inference

In [67]:
prompt = 'Vẫn nhớ những buổi sáng đó'
generator = pipeline('text-generation', model='duydatnguyen/vi-poem-gpt-neo')
results = generator(
    prompt,
    max_new_tokens=300,     # The maximum number of tokens that can be generated (default: 50)
    do_sample=True,         # Whether to sample the next token or use beam search (default: False)
    top_k=50,               # The number of highest probability vocabulary tokens to keep for sampling (default: 50)
    top_p=0.95,             # The cumulative probability of sampling only from the top_k tokens (default: 1.0)
    temperature=0.8,        # Controls the randomness of the generated text (default: 1.0)
    repetition_penalty=1    # Penalty to be applied to sequences to reduce repetition (default: 1.0)
)
results = results[0]['generated_text']
for line in results.split('\n'):
    print(line)

Vẫn nhớ những buổi sáng đó chúng ta sẽ đi tìm một nổi thay
cuộc đời bình yên đến bao giờ sống
một niềm vui không cần buồn chỉ là mình bước lên sóng
mà chúng ta từng nổi
một cuộc đời duy nhất


In [73]:

prompt = 'Đến bao lâu thì sẽ dừng'
generator = pipeline('text-generation', model='duydatnguyen/vi-poem-gpt-neo')
results = generator(
    prompt,
    max_new_tokens=250,
    do_sample=True,
    top_k=60,
    top_p=0.95,
    temperature=1,
    repetition_penalty=1.2
)
results = results[0]['generated_text']
for line in results.split('\n'):
    print(line)

Đến bao lâu thì sẽ dừng lại nhớ
khi người đi qua cũng sẽ bắt đầu một quãng đợi
chước mắt đã biết cách yêu thương không còn phải cứ nhắm mở
vì chưa bao giờ thức nào khác
hãy chuyện cách ta đi
